In [2]:
from google.colab import drive
drive.mount('/content/drive')
import torch

Mounted at /content/drive


In [2]:
!nvidia-smi

Fri Mar 14 09:25:33 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off |   00000000:00:04.0 Off |                    0 |
| N/A   29C    P0             45W /  400W |       0MiB /  40960MiB |      0%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+-----

In [4]:
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device = torch.device("cpu")
print(device)

cpu


Autoprocessing inputs

In [3]:
import pandas as pd
import numpy as np
import torch
from transformers import AutoProcessor


processor = AutoProcessor.from_pretrained("facebook/musicgen-small")

csv_file_path = '/content/drive/MyDrive/cs229Project/229Dataset/musicnet/expanded_musicnet_dataset.csv'
df = pd.read_csv(csv_file_path)

text_prompts = df['text_prompt'].tolist()
audio_paths = df['audio_path'].tolist()

# Fixed length for text sequences
FIXED_TEXT_LENGTH = 30
FIXED_AUDIO_LENGTH = 1920000

inputs = []
for text_prompt, audio_path in zip(text_prompts, audio_paths):
    # Load the audio waveform
    audio_waveform = np.load(audio_path)

    # Process the audio and text
    processed_input = processor(
        text=[text_prompt],
        audio=audio_waveform,
        return_tensors="pt",
        sampling_rate=32000,
        padding=True
    )

    # Reshape `input_ids` and `attention_mask` to `[1, 30]` with padding
    processed_input['input_ids'] = torch.nn.functional.pad(
        processed_input['input_ids'],
        (0, FIXED_TEXT_LENGTH - processed_input['input_ids'].shape[1]),  # Pad to the right
        value=0  # Zero padding for `input_ids`
    ).reshape(1, FIXED_TEXT_LENGTH)

    processed_input['attention_mask'] = torch.nn.functional.pad(
        processed_input['attention_mask'],
        (0, FIXED_TEXT_LENGTH - processed_input['attention_mask'].shape[1]),
        value=0  # Zero padding for `attention_mask`
    ).reshape(1, FIXED_TEXT_LENGTH)

    if processed_input['input_ids'].shape != (1, FIXED_TEXT_LENGTH):
        raise ValueError(f"Unexpected shape for `input_ids`: {processed_input['input_ids'].shape}")

    if processed_input['attention_mask'].shape != (1, FIXED_TEXT_LENGTH):
        raise ValueError(f"Unexpected shape for `attention_mask`: {processed_input['attention_mask'].shape}")

    if processed_input['padding_mask'].shape != (1, FIXED_AUDIO_LENGTH):
        raise ValueError(f"Unexpected shape for `padding_mask`: {processed_input['padding_mask'].shape}")

    if processed_input['input_values'].shape != (1, 1, FIXED_AUDIO_LENGTH):
        raise ValueError(f"Unexpected shape for `input_values`: {processed_input['input_values'].shape}")

    print(f"Processed input shapes: {[v.shape for v in processed_input.values()]}")
    # Append processed input
    inputs.append(processed_input)

print(f"Processed {len(inputs)} samples with `input_ids` and `attention_mask` reshaped to [1, {FIXED_TEXT_LENGTH}].")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/275 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.37k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

Processed input shapes: [torch.Size([1, 30]), torch.Size([1, 30]), torch.Size([1, 1, 1920000]), torch.Size([1, 1920000])]
Processed input shapes: [torch.Size([1, 30]), torch.Size([1, 30]), torch.Size([1, 1, 1920000]), torch.Size([1, 1920000])]
Processed input shapes: [torch.Size([1, 30]), torch.Size([1, 30]), torch.Size([1, 1, 1920000]), torch.Size([1, 1920000])]
Processed input shapes: [torch.Size([1, 30]), torch.Size([1, 30]), torch.Size([1, 1, 1920000]), torch.Size([1, 1920000])]
Processed input shapes: [torch.Size([1, 30]), torch.Size([1, 30]), torch.Size([1, 1, 1920000]), torch.Size([1, 1920000])]
Processed input shapes: [torch.Size([1, 30]), torch.Size([1, 30]), torch.Size([1, 1, 1920000]), torch.Size([1, 1920000])]
Processed input shapes: [torch.Size([1, 30]), torch.Size([1, 30]), torch.Size([1, 1, 1920000]), torch.Size([1, 1920000])]
Processed input shapes: [torch.Size([1, 30]), torch.Size([1, 30]), torch.Size([1, 1, 1920000]), torch.Size([1, 1920000])]
Processed input shapes: 

In [ ]:
save_path = '/content/drive/MyDrive/cs229Project/complete2.pt'

# Save the list of inputs as a tensor
torch.save(inputs, save_path)

print(f"Inputs saved to: {save_path}")

### Load Data

In [ ]:
import torch

# Load the full data onto CPU
inputs = torch.load("/content/drive/MyDrive/cs229Project/complete2.pt", map_location='cpu')

# Check the size of inputs (assuming it's a tensor or a large dataset structure)




<ipython-input-4-694b895fa0d9>:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  inputs = torch.load("/content/drive/MyDrive/cs229Project/complete2.pt", map_location='cpu')


In [ ]:
!nvidia-smi

Fri Mar 14 05:18:01 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off |   00000000:00:04.0 Off |                    0 |
| N/A   29C    P0             44W /  400W |       0MiB /  40960MiB |      0%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+-----

In [5]:
import torch
import torch.nn as nn
import numpy as np
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# Define composer mapping
composer_map = {
    "Schubert": 9, "Mozart": 7, "Dvorak": 4,
    "Cambini": 3, "Haydn": 6, "Brahms": 2,
    "Faure": 5, "Ravel": 8, "Bach": 0, "Beethoven": 1
}

reverse_map = {v: k for k, v in composer_map.items()}

class StyleEncoder(nn.Module):
    def __init__(self, embedding_dim=128, num_composers=10):
        super(StyleEncoder, self).__init__()
        self.cnn = nn.Sequential(
            nn.Conv1d(1, 32, kernel_size=5, stride=2, padding=2),
            nn.ReLU(),
            nn.Conv1d(32, 64, kernel_size=5, stride=2, padding=2),
            nn.ReLU(),
            nn.Conv1d(64, 128, kernel_size=5, stride=2, padding=2),
            nn.ReLU()
        )
        self.temporal = nn.Sequential(
            nn.Conv1d(128, 256, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.AdaptiveAvgPool1d(1)
        )
        self.projection = nn.Linear(256, embedding_dim)
        self.composer_embedding = nn.Embedding(num_composers, embedding_dim)

    def forward(self, x, composer_embedding=None):
        x = self.cnn(x)
        x = self.temporal(x).squeeze(-1)
        x = self.projection(x)
        return x + composer_embedding


# Create the model

style_encoder = StyleEncoder().to(device)

# Load the checkpoint
checkpoint = torch.load("/content/drive/MyDrive/cs229Project/style_encoder-2.pth", map_location=device)
filtered_checkpoint = {k: v for k, v in checkpoint.items() if k in style_encoder.state_dict()}
style_encoder.load_state_dict(filtered_checkpoint, strict=False)
style_encoder.eval()

# Load audio waveform
waveform = torch.tensor(
    np.load('/content/drive/MyDrive/cs229Project/229Dataset/musicnet/processed_audio/1727.npy')
).to(device)
waveform = waveform.to(dtype=torch.float16)
# Make sure waveform has the right shape [batch, channels, time]
if waveform.dim() == 1:
    waveform = waveform.unsqueeze(0).unsqueeze(0)
elif waveform.dim() == 2:
    waveform = waveform.unsqueeze(0)

composer_embedding_weights = style_encoder.composer_embedding.weight.data

composer_embeddings_dict = {
    composer: composer_embedding_weights[composer_id].detach().cpu().numpy()
    for composer, composer_id in composer_map.items()
}
for composer, embedding in composer_embeddings_dict.items():
    print(f"{composer}: {embedding[:5]}...")


print(composer_embeddings_dict["Schubert"])
style_encoder = style_encoder.to(torch.float16)
composer_embeddings = torch.tensor(composer_embeddings_dict["Schubert"]).to(device)
composer_embeddings = composer_embeddings.to(dtype=torch.float16)
print(composer_embeddings.dtype)
print(waveform.dtype)
outputs_with_id = style_encoder(waveform, composer_embeddings)
print("Output shape with composer_id:", outputs_with_id.shape)
print(outputs_with_id)
print(device)



Schubert: [-0.0399006 -1.4581578  0.7540325 -0.3136256 -0.7847596]...
Mozart: [ 0.05876034  2.000688   -1.9079921  -1.1833687  -0.9618606 ]...
Dvorak: [-0.255268    0.17314005 -0.38424298 -1.2285194   0.93682635]...
Cambini: [-0.08229057  1.303195    0.40165764 -1.3665297  -1.1813307 ]...
Haydn: [-1.0088236   1.2965635   0.25496018  0.25001827  0.48092383]...
Brahms: [-0.11956131  0.57687473 -0.14041474 -0.21242541  0.2801856 ]...
Faure: [ 0.06443194  0.12758182  0.17273441 -2.7166872   0.5232171 ]...
Ravel: [-0.37951747 -2.3222933   1.1933309  -0.44496727 -0.07125596]...
Bach: [ 1.0189674  -0.03704567 -0.62848276  0.5494599   0.57478607]...
Beethoven: [-0.22611332  0.53072536 -2.9945054   1.186264   -0.8996345 ]...
[-3.9900597e-02 -1.4581578e+00  7.5403249e-01 -3.1362560e-01
 -7.8475958e-01  2.3423865e-01 -1.7927125e+00 -1.3447340e-01
 -9.9103272e-01  6.0617357e-01 -6.1353147e-01 -6.0587287e-01
  1.0860469e+00  3.6251536e-01  8.9232963e-01  6.0570133e-01
 -1.0701617e+00  6.2439948e-01

In [ ]:
print(device)

cpu


In [12]:
sample_size = 720
random_indices = torch.randperm(2880)[:sample_size]

print(device)
input_ids = (torch.stack([inputs[i]["input_ids"] for i in random_indices])).to(device)
attention_masks = (torch.stack([inputs[i]["attention_mask"] for i in random_indices])).to(device)
padding_masks = (torch.stack([inputs[i]["padding_mask"] for i in random_indices])).to(device)
input_values = (torch.stack([inputs[i]["input_values"] for i in random_indices])).to(device)



print(len(input_ids))
print(len(attention_masks))
print(len(padding_masks))
print(len(input_values))

# del inputs



cuda
720
720
720
720


In [ ]:
del inputs

In [7]:

print(len(input_ids))
print(len(attention_masks))
print(len(padding_masks))
print(len(input_values))


FIXED_TEXT_LENGTH = 30
FIXED_AUDIO_LENGTH = 1920000

for i, attention_mask in enumerate(attention_masks):

    if attention_mask.shape != (1, FIXED_TEXT_LENGTH):
        raise ValueError(
            f"Unexpected shape for `attention_mask` at index {i}: {attention_mask.shape}, expected: (1, {FIXED_TEXT_LENGTH})"
        )

for i, input_id in enumerate(input_ids):
    if input_id.shape != (1, FIXED_TEXT_LENGTH):
        raise ValueError(
            f"Unexpected shape for `input_ids` at index {i}: {input_id.shape}, expected: (1, {FIXED_TEXT_LENGTH})"
        )

for i, padding_mask in enumerate(padding_masks):
    if padding_mask.shape != (1, FIXED_AUDIO_LENGTH):
        raise ValueError(
            f"Unexpected shape for `padding_mask` at index {i}: {padding_mask.shape}, expected: (1, {FIXED_AUDIO_LENGTH})"
        )


for i, input_value in enumerate(input_values):

    # Check input_values shape
    if input_value.shape != (1, 1, FIXED_AUDIO_LENGTH):

        raise ValueError(
            f"Unexpected shape for `input_values` at index {i}: {input_value.shape}, expected: (1, 1, {FIXED_AUDIO_LENGTH})"
        )



100
100
100
100


In [8]:
input_sample = inputs[0]

for key, tensor in input_sample.items():
    print(f"{key}: shape={tensor.shape}, dtype={tensor.dtype}, device={tensor.device}")

input_ids: shape=torch.Size([1, 30]), dtype=torch.int64, device=cpu
attention_mask: shape=torch.Size([1, 30]), dtype=torch.int64, device=cpu
input_values: shape=torch.Size([1, 1, 1920000]), dtype=torch.float16, device=cpu
padding_mask: shape=torch.Size([1, 1920000]), dtype=torch.int32, device=cpu


#No need to run

In [ ]:
import pandas as pd
import numpy as np
import torch

import librosa

def extract_composer_id(text_prompt):
      for composer, composer_id in composer_embeddings_dict.items():
          if f"Convert this classical song to {composer}'s style." in text_prompt:
              # print(f"Detected composer: {composer}, ID: {composer_id}")
              return composer, composer_id
      return None, None

# Path to your CSV file
csv_file_path = '/content/drive/MyDrive/cs229Project/229Dataset/musicnet/expanded_musicnet_dataset.csv'

# Load the CSV file
df = pd.read_csv(csv_file_path)

# Assuming your CSV has two columns: 'text_prompt' and 'audio_path'
text_prompts = df['text_prompt'].tolist()


# List to store the preprocessed inputs
embedding_indices= []
i = 0
# Iterate over the rows of the CSV
for text_prompt in text_prompts:

    composer, composer_id = extract_composer_id(text_prompt)
    if composer_id is not None:
      embedding_indices.append(torch.tensor(composer_id))
      i+=1
      print(composer_id)
    else:
      embedding_indices.append(None)
print(i)
print(len(embedding_indices))
print(len(text_prompts))


save_path = '/content/drive/MyDrive/cs229Project/229Dataset/style_embedding_lists.pt'


# Save the list of inputs as a tensor
torch.save(embedding_indices, save_path)

print(f"embeddings saved to: {save_path}")
# Now `inputs` contains the preprocessed data ready for further use (e.g., model inference)


In [ ]:
print(embedding_indices[0])

tensor([ 0.0588,  2.0007, -1.9080, -1.1834, -0.9619,  1.4126, -1.1918,  1.2700,
         0.7093,  1.1566, -0.7637,  1.7389, -1.6297, -0.5954,  1.5445, -0.0481,
         0.8132, -1.3673,  0.3835, -1.9577,  0.2614, -0.5133,  0.1969,  0.3858,
        -0.8487,  0.6553, -1.6743, -2.1311, -0.5172,  0.8796,  0.8163, -2.2400,
         0.6391,  1.3414,  1.2118,  1.3994, -1.8166,  0.0995, -0.6974, -1.7722,
         0.2349, -0.5651, -1.0648,  0.9417,  0.1981, -0.2291, -0.6043, -1.4199,
         1.2076,  0.9217,  0.1012, -0.8518,  0.3936,  1.3873,  0.9021, -0.1863,
         0.3481, -0.7181,  0.6229,  1.8244,  0.3144, -1.7413, -0.1232,  1.0098,
         0.3884, -1.5974,  0.0151, -1.0418,  2.1074,  1.5174,  0.7467, -0.4092,
         1.2712,  0.2829, -0.9681,  0.6707, -1.8978,  0.3094,  1.2624, -0.2782,
        -0.0503, -0.1144, -1.0810,  0.7620,  0.0531,  0.1957, -2.0264,  0.6840,
         1.1453, -0.9737, -0.2477, -0.3287, -1.3022,  1.3952, -0.8056, -0.2676,
         0.2653, -0.4485, -0.1157,  1.52

In [ ]:
print(device)


cpu


In [ ]:
import gc
gc.collect()


325

# Run


In [ ]:
!pip install --upgrade transformers==4.48.3
!pip install --upgrade peft==0.10.0
!pip install --upgrade torch torchaudio --index-url https://download.pytorch.org/whl/cu118


In [18]:
print(input_values[0].dtype)

torch.float16


In [ ]:
print(device)

cpu


In [38]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from peft import LoraConfig, get_peft_model
from transformers import AutoProcessor, MusicgenForConditionalGeneration
import torchaudio
import numpy as np

device = torch.device("cpu")

from transformers import AutoProcessor, MusicgenForConditionalGeneration
from peft import LoraConfig, get_peft_model

class EnhancedMusicGenerator(nn.Module):
    def __init__(self):
        super().__init__()
        self.style_encoder = StyleEncoder()
        self.style_encoder = self.style_encoder.to(torch.float16)
        self.base_model = MusicgenForConditionalGeneration.from_pretrained(
            "facebook/musicgen-small",
            torch_dtype=torch.float16
        )

        lora_config = LoraConfig(
            r=16,
            lora_alpha=32,
            lora_dropout=0.1,
            bias='none',
            target_modules=["q_proj", "v_proj"]
        )

        self.model = get_peft_model(self.base_model, lora_config).to(device)
        self.processor = AutoProcessor.from_pretrained("facebook/musicgen-small")

        for module in self.model.modules():
            if module.__class__.__name__ == 'MusicgenSinusoidalPositionalEmbedding':
                module.offset = 1



    def get_composer_mean_embeddings(self):
        composer_embedding_weights = self.style_encoder.composer_embedding.weight.data
        return {
            composer: composer_embedding_weights[composer_id].detach().cpu()
            for composer, composer_id in self.composer_map.items()
        }

    def extract_style_embeddings(self, embedding_idx):
        return self.composer_mean_embeddings[reverse_map[embedding_idx]]

    def forward(self, input, embedding_idx, batch_size):
        model_param = next(self.model.parameters())
        model_device = model_param.device

        outputs = []
        for i in range(batch_size):
            processed_input = {key: val[i].to(model_device) for key, val in input.items()}

            processed_input['input_ids'] = processed_input['input_ids'].to(dtype=torch.int32)
            processed_input['attention_mask'] = processed_input['attention_mask'].to(dtype=torch.int32)
            processed_input['input_values'] = processed_input['input_values'].to(dtype=torch.float32)
            processed_input['padding_mask'] = processed_input['padding_mask'].to(dtype=torch.int32)

            model_inputs = processed_input

            try:
                generated_audio = self.model.generate(
                    **model_inputs,
                    guidance_scale=3,
                    max_new_tokens=1500,
                    do_sample=True,
                    temperature=1.0
                )
                outputs.append(generated_audio.to(dtype=torch.float16))
                print(generated_audio.shape)
            except Exception as e:
                print(f"Error during generation: {e}")

        if not outputs:
            raise RuntimeError("No audio was successfully generated.")

        return torch.stack(outputs)


    def compute_loss(self, generated_audio, embedding_idx):
        try:
            total_loss = 0.0
            target_length = 32000 * 30

            for i in range(generated_audio.size(0)):

                audio_sample = generated_audio[i].to(torch.float16)
                embedding = embedding_idx[i]

                if torch.isnan(audio_sample).any():
                    print(f"NaN detected in generated_audio at sample {i}")
                if torch.isnan(embedding).any():
                    print(f"NaN detected in embedding_idx at sample {i}")

                audio_sample = (generated_audio[i]).to(torch.float16)

                embedding = embedding_idx[i]


                length = audio_sample.size(-1)
                if length < target_length:
                    padding = target_length - length
                    audio_sample = F.pad(audio_sample, (0, padding), mode='constant', value=0)
                    noise = torch.randn_like(audio_sample) * 1e-6
                    audio_sample += noise



                generated_embedding = self.style_encoder(audio_sample.squeeze(0), embedding.squeeze(0))


                target_embedding = embedding.squeeze(0)
                scaling_factor = 100
                generated_embedding = generated_embedding * scaling_factor
                target_embedding = target_embedding * scaling_factor


                loss = F.mse_loss(
                    generated_embedding.to(torch.float32),
                    target_embedding.to(torch.float32)
                    )

                total_loss += loss

            # Return the average loss over the batch
            return total_loss / generated_audio.size(0)
        except Exception as e:
            print(f"Error in compute_loss: {e}")
            return torch.tensor(0.0, device=device)

def train(model, train_loader, optimizer, scheduler, device, batch_size):
    model.train()
    total_loss = 0
    model = model.to(device)
    i = 0
    loss_list = []
    for input, embedding_idx in train_loader:
        # Move each tensor in the dictionary to device
        i += 1
        input = {key: value.to(device) for key, value in input.items()}
        embedding_idx = embedding_idx.to(device)

        optimizer.zero_grad()
        generated_audio = model(input, embedding_idx, batch_size)
        loss = model.compute_loss(generated_audio, embedding_idx)

        loss.backward()
        optimizer.step()
        scheduler.step()
        print(f"Loss for sample {i}: {loss.item()}")
        total_loss += loss.item()
        loss_list.append(loss.item())

    return total_loss / len(train_loader), loss

In [20]:
print(device)

cpu


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:


!nvidia-smi

Fri Mar 14 05:22:21 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off |   00000000:00:04.0 Off |                    0 |
| N/A   29C    P0             44W /  400W |       0MiB /  40960MiB |      0%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+-----

In [39]:
# CUDA_LAUNCH_BLOCKING=1
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
model = EnhancedMusicGenerator().to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=0.5, weight_decay=1e-2)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.9)

cuda


Config of the text_encoder: <class 'transformers.models.t5.modeling_t5.T5EncoderModel'> is overwritten by shared text_encoder config: T5Config {
  "_name_or_path": "t5-base",
  "architectures": [
    "T5ForConditionalGeneration"
  ],
  "classifier_dropout": 0.0,
  "d_ff": 3072,
  "d_kv": 64,
  "d_model": 768,
  "decoder_start_token_id": 0,
  "dense_act_fn": "relu",
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "relu",
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "is_gated_act": false,
  "layer_norm_epsilon": 1e-06,
  "model_type": "t5",
  "n_positions": 512,
  "num_decoder_layers": 12,
  "num_heads": 12,
  "num_layers": 12,
  "output_past": true,
  "pad_token_id": 0,
  "relative_attention_max_distance": 128,
  "relative_attention_num_buckets": 32,
  "task_specific_params": {
    "summarization": {
      "early_stopping": true,
      "length_penalty": 2.0,
      "max_length": 200,
      "min_length": 30,
      "no_repeat_ngram_size": 3,
      "num_be

In [ ]:
print(device)

cpu


In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
from torch.utils.data import Subset, DataLoader

from torch.utils.data import Dataset, DataLoader
import torch

class MusicDataset(Dataset):
    def __init__(self, input_ids, attention_masks, padding_masks, input_values, embedding_indices):
        self.input_ids = input_ids
        self.attention_masks = attention_masks
        self.padding_masks = padding_masks
        self.input_values = input_values
        self.embedding_indices = embedding_indices

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return {
            "input_ids": self.input_ids[idx],
            "attention_mask": self.attention_masks[idx],
            "padding_mask": self.padding_masks[idx],
            "input_values": self.input_values[idx]
        }, self.embedding_indices[idx]


def collate_fn(batch):
    batch_inputs = {}  # Dictionary to store all input fields
    embedding_idx_batch = []

    # Extract the first valid sample to get the keys
    first_sample = next((s for s in batch if s[0] is not None), None)
    if first_sample is None:
        raise ValueError("All samples in batch are None!")

    first_inputs, _ = first_sample
    input_keys = first_inputs.keys()  # Dynamically get keys

    # Initialize empty lists for each field
    for key in input_keys:
        batch_inputs[key] = []

    for sample in batch:
        inputs, embedding_idx = sample  # Unpack properly

        if embedding_idx is None:
            embedding_idx = -1  # Placeholder value if None

        embedding_idx_batch.append(torch.tensor(embedding_idx, dtype=torch.long))

        for key in input_keys:
            tensor = inputs[key]

            # Ensure tensor is 2D
            if tensor.dim() == 1:
                tensor = tensor.unsqueeze(0)

            # Find max sequence length for this key in batch
            max_length = max(s[0][key].shape[-1] for s in batch if s[0] is not None)

            # Apply padding to match max length
            padded_tensor = torch.nn.functional.pad(tensor, (0, max_length - tensor.shape[-1]), value=0)
            batch_inputs[key].append(padded_tensor)

    # Convert lists to tensors
    for key in batch_inputs:
        batch_inputs[key] = torch.stack(batch_inputs[key])
        print(batch_inputs[key])

    embedding_idx_batch = torch.stack(embedding_idx_batch).to(device)

    return batch_inputs, embedding_idx_batch


class MusicgenSinusoidalPositionalEmbedding(nn.Module):
    def __init__(self, num_positions: int, embedding_dim: int):
        super().__init__()
        self.embedding_dim = embedding_dim
        self.offset = 1
        self.make_weights(num_positions, embedding_dim)

# Example usage for the training loop:
if __name__ == "__main__":
    # Preprocessed inputs and composer indices
    embedding_indices = torch.load('/content/drive/MyDrive/cs229Project/229Dataset/style_embedding_lists.pt')
    embedding_indices = (torch.stack([embedding_indices[i] for i in random_indices])).to(device)
    # print(embedding_indices[0])

    # Create dataset
    dataset = MusicDataset(input_ids[:100], attention_masks[:100], padding_masks[:100], input_values[:100], embedding_indices[:100])

    # Create DataLoader
    batch_size = 1  # Change as needed
    train_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)

    # Example usage: Fetch a batch
    for batch_inputs, batch_embedding_indices in train_loader:
        print(f"Batch input shapes: {[batch_inputs[key].shape for key in batch_inputs]}")
        print(f"Batch embedding indices shape: {batch_embedding_indices.shape}")
        break  # Just print the first batch

   # Ensure all input tensors are on the same device

    # Training loop
    for epoch in range(1):
        loss, loss_list = train(model, train_loader, optimizer, scheduler, device, batch_size)
        print(f"Epoch {epoch+1}, Loss: {loss:.4f}")

    # Save the trained model
    torch.save(model.state_dict(), "/content/drive/MyDrive/enhanced_musicgen_peft.pth")


In [42]:
    torch.save(model.state_dict(), "/content/drive/MyDrive/enhanced_musicgen_peft.pth")


In [ ]:
test_input = {
    "input_ids": input_ids[0].unsqueeze(0),
    "attention_mask": attention_masks[0].unsqueeze(0),
    "padding_mask": padding_masks[0].unsqueeze(0),
    "input_values": input_values[0].unsqueeze(0),
}



In [ ]:

embedding = np.zeros(128)
outputs = model.forward(test_input, embedding, 1)

Processed input shapes: [(torch.Size([1, 30]), torch.int32), (torch.Size([1, 30]), torch.int32), (torch.Size([1, 1920000]), torch.int32), (torch.Size([1, 1, 1920000]), torch.float32)]
Error during generation: index out of range in self


RuntimeError: No audio was successfully generated.

In [ ]:
model = MusicgenForConditionalGeneration.from_pretrained("facebook/musicgen-small")

Config of the text_encoder: <class 'transformers.models.t5.modeling_t5.T5EncoderModel'> is overwritten by shared text_encoder config: T5Config {
  "_name_or_path": "t5-base",
  "architectures": [
    "T5ForConditionalGeneration"
  ],
  "classifier_dropout": 0.0,
  "d_ff": 3072,
  "d_kv": 64,
  "d_model": 768,
  "decoder_start_token_id": 0,
  "dense_act_fn": "relu",
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "relu",
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "is_gated_act": false,
  "layer_norm_epsilon": 1e-06,
  "model_type": "t5",
  "n_positions": 512,
  "num_decoder_layers": 12,
  "num_heads": 12,
  "num_layers": 12,
  "output_past": true,
  "pad_token_id": 0,
  "relative_attention_max_distance": 128,
  "relative_attention_num_buckets": 32,
  "task_specific_params": {
    "summarization": {
      "early_stopping": true,
      "length_penalty": 2.0,
      "max_length": 200,
      "min_length": 30,
      "no_repeat_ngram_size": 3,
      "num_be

In [ ]:
test_input = {
    "input_ids": input_ids[0],
    "attention_mask": attention_masks[0],
    "padding_mask": padding_masks[0],
    "input_values": input_values[0],
}

In [ ]:
print(test_input[key][:5] for key in test_input.keys())

<generator object <genexpr> at 0x78b6103284a0>


In [ ]:
# Save as a PyTorch tensor file
torch.save(test_input, "test_input.pt")

# Download the file to your local machine
from google.colab import files
files.download("test_input.pt")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
!pip install --upgrade --quiet pip
!pip install --upgrade --quiet transformers datasets[audio]
from transformers import MusicgenForConditionalGeneration

model = MusicgenForConditionalGeneration.from_pretrained("facebook/musicgen-small")

/usr/local/lib/python3.11/dist-packages/transformers/models/encodec/modeling_encodec.py:124: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.register_buffer("padding_total", torch.tensor(kernel_size - stride, dtype=torch.int64), persistent=False)
Config of the text_encoder: <class 'transformers.models.t5.modeling_t5.T5EncoderModel'> is overwritten by shared text_encoder config: T5Config {
  "_name_or_path": "t5-base",
  "architectures": [
    "T5ForConditionalGeneration"
  ],
  "classifier_dropout": 0.0,
  "d_ff": 3072,
  "d_kv": 64,
  "d_model": 768,
  "decoder_start_token_id": 0,
  "dense_act_fn": "relu",
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "relu",
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "is_gated_act": false,
  "layer_norm_epsilon": 1e-06,
  "model_type": "t5",
  "n_positions": 5